In [1]:
import torch
import numpy as np
import time
from open3d.ml.torch.ops import knn_search

# Set the random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Generate a random point cloud with 1 million points in 3D space
num_points = 1_000_000
dim = 3
points = torch.rand(num_points, dim).float().cuda()

# Set the number of nearest neighbors to find
k = 20

# Use the same points as queries
queries = points

# Warm-up run (to ensure any initialization is done before timing)
_ = knn_search(points, queries, k)

# Perform the benchmark
num_runs = 3  # Reduced from 10 due to increased computation time
total_time = 0

for i in range(num_runs):
    start_time = time.time()
    
    # Perform kNN search
    indices, distances = knn_search(points, queries, k)
    
    # Ensure all GPU operations are completed
    torch.cuda.synchronize()
    
    end_time = time.time()
    run_time = end_time - start_time
    total_time += run_time
    print(f"Run {i+1} time: {run_time:.4f} seconds")

# Calculate average time
avg_time = total_time / num_runs

print(f"\nAverage time for kNN search on {num_points} points with k={k}: {avg_time:.4f} seconds")
print(f"Number of query points: {num_points}")
print(f"Shape of indices: {indices.shape}")
print(f"Shape of distances: {distances.shape}")

# Verify that the first nearest neighbor is always the point itself
first_nn = indices[:, 0]
assert torch.all(first_nn == torch.arange(num_points).to(indices.device)), "First nearest neighbor should be the point itself"
print("Verification passed: First nearest neighbor is always the point itself")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Exception: Version mismatch: Open3D needs PyTorch version 1.13.*, but version 1.12.1 is installed!